In [1]:
%cd ..

C:\Users\mgrinberg\Desktop\disc c\SDPC\SDPC\SDPC


In [2]:
import sqlalchemy
from time import strftime
from variables import *

In [3]:
def read_lst(lst_path, lst_cols):
    """Cargar el .lst a un DataFrame"""
    return pd.read_fwf(lst_path, encoding='cp1252', widths= [len(x) for x in lst_cols.split('|')])

In [4]:
def get_ge(x):
    """Mapear Grupos Economicos a un indice"""
    for g in grupos:
        if g in x:
            return grupos.index(g)
    else:
        return -1

In [5]:
def preprocesamiento(df):
    """Toma DataFrame y devuelve tensor Q"""
    from variables import c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col, q_col, dev_col
    
    # Tranformo a numerico
    df[[q_col, dev_col]] = df[[q_col, dev_col]].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'))
    df[[c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col]] = df[[c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col]].apply(pd.to_numeric, args=(dict(errors= 'coerce')))
    
    # Saco nulos y el ultimo que era '\x00\x00\x00\x00\x00\x00\x00\x00'
    df = df[~df.isnull()]
    df = df[:-1]
    
    df = df.rename(columns={'Regi¢n':'Region'})

    # Calculo venta bruta 
    df[neto_col] = df[q_col] - df[dev_col]

    # Se borran ventas netas negativas
    df = df[df[neto_col] > 0]

    return df

In [6]:
def lst_to_sql(lst_path, lst_cols, table_name, n=0, mode='append'):
    print('Leyendo lst.')
    df = read_lst(lst_path, lst_cols)
    print('Preprocesando lst.')
    df = preprocesamiento(df)
    df['Grupo Economico'] = df[nom_cli_col].map(get_ge)
    df['Fecha Base'] = strftime("%Y-%m-%d")
    df['N Base'] = n
    print('Subiendo lst a SQL.')
    engine = sqlalchemy.create_engine(f"mssql+pyodbc://{UID}:{PWD}@{SERVER}/{DATABASE}?driver=SQL+Server")
    df.to_sql(table_name, con=engine, if_exists='append')